In [ ]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')
!git clone https://github.com/BeileiCui/EndoDAC

req_path = "/content/EndoDAC/requirements.txt"
if os.path.exists(req_path):
    with open(req_path, "r") as f:
        lines = f.readlines()
    with open(req_path, "w") as f:
        for line in lines:
            if "torch" in line or "torchvision" in line or "xformers" in line:
                print(f"   Skipping: {line.strip()}")
                continue
            f.write(line)

!pip install -q xformers --no-deps
!pip install -q tensorboardX timm gdown fvcore
!pip install -q -r /content/EndoDAC/requirements.txt


Mounted at /content/drive
Cloning into 'EndoDAC'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 235 (delta 60), reused 2 (delta 0), pack-reused 119 (from 1)
Receiving objects: 100% (235/235), 1.43 MiB | 19.22 MiB/s, done.
Resolving deltas: 100% (85/85), done.
   Skipping: --extra-index-url https://download.pytorch.org/whl/cu117
   Skipping: torch==2.0.0
   Skipping: torchvision==0.15.0
   Skipping: torchmetrics==0.10.3
   Skipping: xformers==0.0.18
✅ Patched requirements.txt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
import os
import glob
import zipfile
import gdown
from tqdm import tqdm

repo_path = "/content/EndoDAC"

# Pretrained Model
pretrained_dir = os.path.join(repo_path, "pretrained_model")
pretrained_file = os.path.join(pretrained_dir, "depth_anything_vitb14.pth")

if not os.path.exists(pretrained_file):
    print("Loading Depth-Anything pretrained model...")
    os.makedirs(pretrained_dir, exist_ok=True)
    file_id = "163ILZcnz_-IUoIgy1UF_r7PAQBqgDbll"
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, pretrained_file, quiet=False)
    print(f" Pretrained model: {pretrained_file}")
else:
    print(f" Pretrained model exists")

train_zip_path = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBufferS2S8.zip"
training_data_root = "/content/training_data_simcol"

if not os.path.exists(training_data_root):
    print(f"⏳ Đang giải nén TRAINING DATA từ {train_zip_path}...")
    os.makedirs(training_data_root, exist_ok=True)

    with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
        zip_ref.extractall(training_data_root)

    subfolders = [f.path for f in os.scandir(training_data_root) if f.is_dir()]
    print(f"📂 Sequences: {[os.path.basename(f) for f in subfolders]}")

    # Correct file name: FrameBuffer_0000.png -> 0000000000.png
    print("⏳ Đang chuẩn hóa tên file...")
    for folder in subfolders:
        images = glob.glob(os.path.join(folder, "*.png"))
        for img_path in tqdm(images, desc=f"Renaming {os.path.basename(folder)}"):
            folder_path, filename = os.path.split(img_path)
            if "FrameBuffer_" in filename:
                new_name = filename.replace("FrameBuffer_", "")
                try:
                    num = int(new_name.split('.')[0])
                    new_filename = "{:010d}.png".format(num)
                    os.rename(img_path, os.path.join(folder_path, new_filename))
                except ValueError:
                    pass

    print(f"Data prepared at: {training_data_root}")
else:
    print(f"Data existed: {training_data_root}")

⏳ Đang tải Depth-Anything pretrained model...


Downloading...
From (original): https://drive.google.com/uc?id=163ILZcnz_-IUoIgy1UF_r7PAQBqgDbll
From (redirected): https://drive.google.com/uc?id=163ILZcnz_-IUoIgy1UF_r7PAQBqgDbll&confirm=t&uuid=39937089-18a9-4ff2-b2ef-457f30a3bdd3
To: /content/EndoDAC/pretrained_model/depth_anything_vitb14.pth
100%|██████████| 390M/390M [00:03<00:00, 112MB/s]


✅ Pretrained model: /content/EndoDAC/pretrained_model/depth_anything_vitb14.pth
⏳ Đang giải nén TRAINING DATA từ /content/drive/MyDrive/3D_reconstruction/datasets/FrameBufferS2S8.zip...
📂 Sequences: ['FrameBuffer_S4', 'FrameBuffer_S5', 'FrameBuffer_S3', 'FrameBuffer_S7', 'FrameBuffer_S8', 'FrameBuffer_S6', 'FrameBuffer_S2']
⏳ Đang chuẩn hóa tên file...


Renaming FrameBuffer_S2: 100%|██████████| 1201/1201 [00:00<00:00, 32518.60it/s]

✅ Dữ liệu sẵn sàng tại: /content/training_data_simcol


In [ ]:
import os
import glob
import numpy as np
import torch

repo_path = "/content/EndoDAC"
training_data_root = "/content/training_data_simcol"

#(utils/layers.py)

layers_path = os.path.join(repo_path, "utils", "layers.py")
with open(layers_path, 'r') as f:
    l_content = f.read()

unsafe_code = """    values[invalid] = 0

    corresponding_map.scatter_add_(1, indices, values)"""

safe_code = """    values[invalid] = 0
    indices[invalid] = 0
    indices = indices.clamp(0, H * W - 1)

    corresponding_map.scatter_add_(1, indices, values)"""

if unsafe_code in l_content:
    l_content = l_content.replace(unsafe_code, safe_code)
    with open(layers_path, 'w') as f:
        f.write(l_content)
    print("Fix crash at utils/layers.py")
else:
    print("utils/layers.py had fixed")

# Create SimColDataset CLASS
# Normalized K
simcol_dataset_content = '''from __future__ import absolute_import, division, print_function

import os
import numpy as np
import PIL.Image as pil

from .mono_dataset import MonoDataset

class SimColDataset(MonoDataset):
    """Custom dataset for SimCol3D data
    Image format: 0000000000.png (10 digits)
    """
    def __init__(self, *args, **kwargs):
        super(SimColDataset, self).__init__(*args, **kwargs)

        W_orig = 475.0
        H_orig = 475.0

        # Intrinsic
        fx_orig = 227.60416
        fy_orig = 237.5
        cx_orig = 227.60416
        cy_orig = 237.5

        fx_norm = fx_orig / W_orig
        fy_norm = fy_orig / H_orig
        cx_norm = cx_orig / W_orig
        cy_norm = cy_orig / H_orig

        self.K = np.array([[fx_norm, 0., cx_norm, 0.],
                           [0., fy_norm, cy_norm, 0.],
                           [0., 0., 1., 0.],
                           [0., 0., 0., 1.]], dtype=np.float32)

        self.side_map = {"l": "left", "r": "right"}

    def check_depth(self):
        return False

    def get_image_path(self, folder, frame_index, side):
        f_str = "{:010d}.png".format(frame_index)
        return os.path.join(self.data_path, folder, f_str)

    def get_color(self, folder, frame_index, side, do_flip):
        image_path = self.get_image_path(folder, frame_index, side)
        color = self.loader(image_path)
        if do_flip:
            color = color.transpose(pil.FLIP_LEFT_RIGHT)
        return color
'''

with open(os.path.join(repo_path, "datasets", "simcol_dataset.py"), "w") as f:
    f.write(simcol_dataset_content)
print("Created datasets/simcol_dataset.py")

# Update __init__.py
init_path = os.path.join(repo_path, "datasets", "__init__.py")
init_content = '''from .mono_dataset import MonoDataset
from .scared_dataset import SCAREDDataset, SCAREDRAWDataset
from .hamlyn_dataset import HamlynDataset
from .c3vd_dataset import C3VDDataset
from .simcol_dataset import SimColDataset
'''
with open(init_path, "w") as f:
    f.write(init_content)

# Update trainer_end_to_end.py
trainer_path = os.path.join(repo_path, "trainer_end_to_end.py")
with open(trainer_path, "r") as f:
    t_content = f.read()
if 'datasets_dict = {"endovis": datasets.SCAREDRAWDataset}' in t_content:
    t_content = t_content.replace(
        'datasets_dict = {"endovis": datasets.SCAREDRAWDataset}',
        'datasets_dict = {"endovis": datasets.SCAREDRAWDataset, "simcol": datasets.SimColDataset}'
    )
    with open(trainer_path, "w") as f:
        f.write(t_content)
print(" SimColDataset update to Trainer")

# Update options.py
options_path = os.path.join(repo_path, "options.py")
with open(options_path, "r") as f:
    o_content = f.read()
if 'choices=["endovis", "simcol"]' not in o_content:
    o_content = o_content.replace('choices=["endovis"]', 'choices=["endovis", "simcol"]')
if '"simcol"' not in o_content:
    o_content = o_content.replace('"hamlyn", "c3vd", "endovis"', '"hamlyn", "c3vd", "endovis", "simcol"')
with open(options_path, "w") as f:
    f.write(o_content)

# Fix error mono_dataset.py
mono_path = os.path.join(repo_path, "datasets", "mono_dataset.py")
with open(mono_path, "r") as f:
    m_content = f.read()
if "sequence = folder[7]" in m_content:
    safe_parse = '''
        try:
            if "_" in folder and folder.split("_")[-1].isdigit():
                seq_id = int(folder.split("_")[-1])
            elif folder.isdigit():
                seq_id = int(folder)
            else:
                seq_id = 0
        except:
            seq_id = 0
        inputs["sequence"] = torch.from_numpy(np.array(seq_id))
        inputs["keyframe"] = torch.from_numpy(np.array(0))'''

    lines = m_content.splitlines()
    new_lines = []
    skip = 0
    for line in lines:
        if "sequence = folder[7]" in line:
            new_lines.append(safe_parse)
            skip = 3
        elif skip > 0:
            skip -= 1
        else:
            new_lines.append(line)
    with open(mono_path, "w") as f:
        f.write("\n".join(new_lines))
    print("Fix parsing sequence at mono_dataset.py")

# SPLIT FILES

split_dir = os.path.join(repo_path, "splits", "simcol")
os.makedirs(split_dir, exist_ok=True)

sequence_folders = sorted([f.name for f in os.scandir(training_data_root) if f.is_dir()])
all_train, all_val = [], []

print(f"Split files from {len(sequence_folders)}")

for seq_name in sequence_folders:
    seq_path = os.path.join(training_data_root, seq_name)
    images = glob.glob(os.path.join(seq_path, "*.png"))
    indices = sorted([int(os.path.basename(img).split('.')[0]) for img in images])

    if len(indices) < 5: continue

    valid_indices = indices[2:-2]
    split_idx = int(len(valid_indices) * 0.9)

    for idx in valid_indices[:split_idx]:
        all_train.append(f"{seq_name}\t{idx}\tl\n")
    for idx in valid_indices[split_idx:]:
        all_val.append(f"{seq_name}\t{idx}\tl\n")

with open(os.path.join(split_dir, "train_files.txt"), "w") as f: f.writelines(all_train)
with open(os.path.join(split_dir, "val_files.txt"), "w") as f: f.writelines(all_val)
with open(os.path.join(split_dir, "test_files.txt"), "w") as f: f.writelines(all_val)

# Dummy GT depth
dummy_depths = [np.zeros((256, 320)) for _ in range(len(all_val))]
np.savez(os.path.join(split_dir, "gt_depths.npz"), data=dummy_depths)

print(f"Split files creates at: {split_dir}")
print(f"Train: {len(all_train)} | Val: {len(all_val)}")

✅ Đã vá lỗi crash trong utils/layers.py (Safety Patch Applied)
✅ Tạo datasets/simcol_dataset.py (Với Normalized K chuẩn)
✅ Đã đăng ký SimColDataset vào Trainer
✅ Đã sửa lỗi parsing sequence trong mono_dataset.py
🔄 Tạo split files từ 7 thư mục...
✅ Split files tạo xong tại: /content/EndoDAC/splits/simcol
📊 Train: 7539 | Val: 840


In [ ]:
import os
import sys

os.environ["COLAB_TPU_ADDR"] = ""

repo_path = "/content/EndoDAC"
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

model_name = "endodac_simcol_v1"
log_dir = "/content/drive/MyDrive/3D_reconstruction/outputs/EndoDAC"
data_path = "/content/training_data_simcol"

os.makedirs(log_dir, exist_ok=True)

!cd "{repo_path}" && python train_end_to_end.py \
  --data_path "{data_path}" \
  --model_name "{model_name}" \
  --split "simcol" \
  --eval_split "simcol" \
  --dataset "simcol" \
  --batch_size 8 \
  --learning_rate 1e-5 \
  --num_epochs 20 \
  --height 256 \
  --width 320 \
  --log_dir "{log_dir}" \
  --frame_ids 0 -1 1 \
  --min_depth 1 \
  --max_depth 23 \
  --lora_type "dvlora" \
  --lora_rank 4 \
  --learn_intrinsics False \
  --num_workers 2

/content/EndoDAC/models/backbones/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/content/EndoDAC/models/backbones/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/content/EndoDAC/models/backbones/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
load pretrained weight from /content/EndoDAC/pretrained_model/depth_anything_vitb14.pth

Training model named:
   endodac_simcol_v1
Models and tensorboard events files are saved to:
   /content/drive/MyDrive/3D_reconstruction/outputs/EndoDAC
Training is using:
   cuda
/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4317.)
  return _VF.meshgrid(tenso

## 5. Inference & Visualization

In [ ]:
import os
import glob
import zipfile
from tqdm import tqdm

test_zip_path = "/content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip"
test_extract_path = "/content/test_data_s1"

if not os.path.exists(test_extract_path):
    print(f"⏳ Đang giải nén S1 test data từ {test_zip_path}...")
    os.makedirs(test_extract_path, exist_ok=True)

    with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
        zip_ref.extractall(test_extract_path)

    print(f"Test data at: {test_extract_path}")
else:
    print(f"Test data has exist: {test_extract_path}")

test_images_dir = None
for root, dirs, files in os.walk(test_extract_path):
    if any(f.endswith('.png') for f in files):
        test_images_dir = root
        break

if test_images_dir:
    test_images = sorted(glob.glob(os.path.join(test_images_dir, "*.png")))
    print(f"Test images: {test_images_dir}")
    print(f"frames: {len(test_images)}")
else:
    print("No frames in file zip!")

⏳ Đang giải nén S1 test data từ /content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip...
✅ Test data S1 sẵn sàng tại: /content/test_data_s1
📂 Test images: /content/test_data_s1/FrameBuffer_RGB
📊 Số lượng frames: 1201


In [ ]:
repo_path = "/content/EndoDAC"
model_name = "endodac_simcol_v1"
log_dir = "/content/drive/MyDrive/3D_reconstruction/outputs/EndoDAC"
test_images_dir = "/content/test_data_s1/FrameBuffer_RGB"
output_base = "/content/drive/MyDrive/3D_reconstruction/inference_endodac_S1"

lora_rank = 4
lora_type = "dvlora"
min_depth = 1
max_depth = 23
height = 256
width = 320

In [ ]:

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import glob
import os
import sys

if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

import models.endodac as endodac
import models.encoders as encoders
import models.decoders as decoders
from utils.layers import disp_to_depth, transformation_from_parameters
from torchvision import transforms


weights_folder = os.path.join(log_dir, model_name, "models", "weights_last")
if not os.path.exists(weights_folder):
    weights_folder = os.path.join(log_dir, model_name, "models", "weights_epoch")
print(f"Loading weights: {weights_folder}")

# Load Depth Model
print("Loading Depth Model")
depth_model = endodac.endodac(
    backbone_size="base",
    r=lora_rank,
    lora_type=lora_type,
    image_shape=(224, 280),
    pretrained_path=os.path.join(repo_path, "pretrained_model"),
    residual_block_indexes=[2, 5, 8, 11],
    include_cls_token=True
)

depth_weights_path = os.path.join(weights_folder, "depth_model.pth")
depth_weights = torch.load(depth_weights_path, map_location=device)
model_dict = depth_model.state_dict()
depth_model.load_state_dict({k: v for k, v in depth_weights.items() if k in model_dict})
depth_model.to(device)
depth_model.eval()
print("Depth Model loaded!")

# Load Pose Model
print("Loading Pose Model")
pose_encoder = encoders.ResnetEncoder(18, False, num_input_images=2)
pose_decoder = decoders.PoseDecoder(pose_encoder.num_ch_enc, num_input_features=1, num_frames_to_predict_for=2)

pose_enc_path = os.path.join(weights_folder, "pose_encoder.pth")
pose_dec_path = os.path.join(weights_folder, "pose.pth")

if os.path.exists(pose_enc_path) and os.path.exists(pose_dec_path):
    pose_encoder.load_state_dict(torch.load(pose_enc_path, map_location=device))
    pose_decoder.load_state_dict(torch.load(pose_dec_path, map_location=device))
    pose_encoder.to(device)
    pose_decoder.to(device)
    pose_encoder.eval()
    pose_decoder.eval()
    print("Pose Model loaded!")
else:
    print("No Pose model ")
    pose_encoder = None
    pose_decoder = None

/content/EndoDAC/models/backbones/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/content/EndoDAC/models/backbones/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/content/EndoDAC/models/backbones/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")


🖥️ Device: cuda
📂 Loading weights từ: /content/drive/MyDrive/3D_reconstruction/outputs/EndoDAC/endodac_simcol_v1/models/weights_last
⏳ Loading Depth Model...
load pretrained weight from /content/EndoDAC/pretrained_model/depth_anything_vitb14.pth

✅ Depth Model loaded!
⏳ Loading Pose Model...
✅ Pose Model loaded!


In [ ]:

depth_output_dir = os.path.join(output_base, "depth")
os.makedirs(depth_output_dir, exist_ok=True)

test_images = sorted(glob.glob(os.path.join(test_images_dir, "*.png")))
print(f"Processing {len(test_images)} frames...")

to_tensor = transforms.ToTensor()
poses = [np.eye(4)]
current_pose = np.eye(4)

with torch.no_grad():
    for i in tqdm(range(len(test_images))):
        # Load and preprocess image
        img_pil = Image.open(test_images[i]).convert('RGB')
        img_resized = img_pil.resize((width, height), Image.LANCZOS)
        img_tensor = to_tensor(img_resized).unsqueeze(0).to(device)

        # DEPTH PREDICTION
        outputs = depth_model(img_tensor)
        disp = outputs[("disp", 0)]
        disp_np = disp[0, 0].cpu().numpy()
        # Convert Disparity to Depth
        depth_np = 1.0 / np.maximum(disp_np, 1e-7)
        # .npy
        np.save(os.path.join(depth_output_dir, f"{i:010d}.npy"), depth_np)
        # VISUALIZATION DEPTH
        #plt.imsave(
            #os.path.join(depth_output_dir, f"{i:010d}.png"),
            #depth_np,
            #cmap='plasma'
        #)

        # Lưu .npy (giá trị thô)
        np.save(os.path.join(depth_output_dir, f"{i:010d}.npy"), disp_np)
        # VISUALIZATION DEPTH
        plt.imsave(
            os.path.join(depth_output_dir, f"{i:010d}.png"),
            disp_np,
            cmap='magma'
        )

        # POSE PREDICTION
        if pose_encoder is not None and i < len(test_images) - 1:
            img_next_pil = Image.open(test_images[i + 1]).convert('RGB')
            img_next_resized = img_next_pil.resize((width, height), Image.LANCZOS)
            img_next_tensor = to_tensor(img_next_resized).unsqueeze(0).to(device)

            pose_input = torch.cat([img_next_tensor, img_tensor], dim=1)
            features = [pose_encoder(pose_input)]
            axisangle, translation, _ = pose_decoder(features)

            pose_mat = transformation_from_parameters(
                axisangle[:, 0], translation[:, 0]
            ).cpu().numpy()[0]

            current_pose = current_pose @ pose_mat
            poses.append(current_pose.copy())

print(f"\n Depth maps saved to: {depth_output_dir}")
print(f"Total frames: {len(test_images)}")

📊 Processing 1201 frames...


 14%|█▎        | 164/1201 [02:40<14:16,  1.21it/s]

In [ ]:
# SAVE TRAJECTORY
trajectory_path = os.path.join(output_base, "trajectory_S1.txt")
with open(trajectory_path, "w") as f:
    for pose in poses:
        row_str = " ".join(map(str, pose[:3, :].flatten()))
        f.write(row_str + "\n")

npz_path = os.path.join(output_base, "poses_S1.npz")
np.savez_compressed(npz_path, poses=np.array(poses))

print(f" Trajectory saved to: {trajectory_path}")
print(f" Poses saved to: {npz_path}")
print(f" Total poses: {len(poses)}")